In [20]:

import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk

nltk.download('punkt')

#preprocessing the text

def preprocess_text(text):
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    return inputs['input_ids'], inputs['attention_mask']

def get_embeddings(text):
    input_ids, attention_mask = preprocess_text(text)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    return outputs.last_hidden_state

def extractive_summarization(text, num_sentences=1):
    sentences = nltk.sent_tokenize(text)
    sentence_embeddings = [get_embeddings(sentence).numpy().mean(axis=1) for sentence in sentences]
    text_embedding = get_embeddings(text).numpy().mean(axis=1)
    
    similarities = [cosine_similarity(text_embedding, sentence_embedding)[0][0] for sentence_embedding in sentence_embeddings]
    
    ranked_sentences = [sentences[i] for i in np.argsort(similarities)[-num_sentences:]]
    return ' '.join(ranked_sentences)

from transformers import DistilBertTokenizer, DistilBertModel

# Load tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')





text = """A Dog, to whom the butcher had thrown a bone, was hurrying home with his prize as fast as he could go. As he crossed a narrow footbridge,
 he happened to look down and saw himself reflected in the quiet water as if in a mirror. But the greedy Dog thought he saw a real Dog carrying a bone
 much bigger than his own. If he had stopped to think he would have known better. But instead of thinking, he dropped his bone and sprang at the Dog in the river,
 only to find himself swimming for dear life to reach the shore. At last he managed to scramble out, and as he stood sadly thinking about the good bone he had lost,
 he realized what a stupid Dog he had been."""
summary = extractive_summarization(text)
print(summary)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gopivardhangunta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


At last he managed to scramble out, and as he stood sadly thinking about the good bone he had lost,
 he realized what a stupid Dog he had been.
